In [13]:
import gzip
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kyeling\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
fpath = "../data/beeradvocate.json.gz"

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

data = []
for l in tqdm(readGz(fpath)):
    data.append(l)
    if len(data) >= 50000:
        break

49999it [00:03, 14370.48it/s]


In [23]:
np.random.shuffle(data)
n = len(data)

dataTrain = data[:int(n*0.8)]
dataVal = data[int(n*0.8):int(n*0.9)] 
dataTest = data[int(n*0.9):] 

Xtrain = [d['review/text'] for d in dataTrain]
ytrain = [d['beer/style'] for d in dataTrain]

Xval = [d['review/text'] for d in dataVal]
yval   = [d['beer/style'] for d in dataVal]

In [25]:
# count_vect = CountVectorizer()
# Xtrain = count_vect.fit_transform([d['review/text'] for d in dataTrain])
# print(count_vect.vocabulary_.get(u'algorithm'))

# tfidf_vect = TfidfVectorizer()
# Xtrain = tfidf_vect.fit_transform([d['review/text'] for d in dataTrain])

In [31]:
bow_model = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression()),
])

bow_params = {
    'clf__C': np.arange(1,11)
}

tfidf_model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])

tfidf_params = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__C': np.arange(1,11)
}


In [ ]:
best_bow = GridSearchCV(bow_model, bow_params, cv=5, n_jobs=-1)

In [ ]:
best_tfidf = GridSearchCV(tfidf_model, tfidf_params, cv=5, n_jobs=-1)

In [32]:
# text_clf.fit([d['review/text'] for d in dataTrain], [d['beer/style'] for d in dataTrain])
# preds = text_clf.predict([d['review/text'] for d in dataVal])

# correct = [int(yp == y) for yp, y in zip(preds, yval)]
# acc = sum(correct) / len(correct)

c:\Users\kyeling\Desktop\CSE258_RecommenderSystems\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
